### Estimación de peso y dimensiones de los envíos de Mercado Libre

Este notebook es una versión mínima y necesaria para poder levantar el dataset [meli_dataset_20190426.csv](https://drive.google.com/open?id=1tNUKD1lf1z8C7LPpCruiDl6SKBAHb79v) y comprender los features que éste contiene.

#### Sobre el dataset
El dataset corresponde a un muestreo de envíos durante tres meses de 2018 en MLB (Mercado Libre Brasil), filtrados según métodos de envíos y por carriers (empresas de correos a cargo del envío propiamente dicho). A su vez, fue muestreado de forma no uniforme para romper de forma manual la distribución de los datos.

In [1]:
import pandas as pd
import random

In [2]:
random.seed(0)

In [3]:
DATASET = 'meli_dataset_20190426.csv'

In [4]:
df = pd.read_csv(DATASET, low_memory=False)

In [5]:
df.columns

Index(['ITEM_ID', 'SHP_WEIGHT', 'SHP_LENGTH', 'SHP_WIDTH', 'SHP_HEIGHT',
       'ATTRIBUTES', 'CATALOG_PRODUCT_ID', 'CONDITION', 'DOMAIN_ID', 'PRICE',
       'SELLER_ID', 'STATUS', 'TITLE'],
      dtype='object')

#### ¿Qué significa cada columna?

* ITEM_ID: id unívoco de cada item publicado. (Ofuscado)
* SHP_WEIGHT: peso del paquete informado por el correo.
* SHP_LENGTH: largo del paquete informado por el correo.
* SHP_WIDTH: ancho del paquete informado por el correo.
* SHP_HEIGHT: altura del paquete informado por el correo.
* ATTRIBUTES: atributos como marca y modelo, entre otros, en formato json-lines
* CATALOG_PRODUCT_ID: id del catálogo (ofuscado).
* CONDITION: condición de venta (nuevo o usado).
* DOMAIN_ID: id de la categoría a la que pertenece la publicación.
* PRICE: precio en reales.
* SELLER_ID: id del vendedor (ofuscado).
* STATUS: estado de la publicación (activa, cerrada, pausada, etc.)
* TITLE: título de la publicación.

In [6]:
df.sample(10)

,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,ATTRIBUTES,CATALOG_PRODUCT_ID,CONDITION,DOMAIN_ID,PRICE,SELLER_ID,STATUS,TITLE
393182,DOBMLTD4EA,510.0,16.0,11.0,2.0,NaN,H53U1H7Q5G,NaN,NaN,NaN,QMHWF5HDJV,under_review,Kimera 2 Unidades X 60 Cápsulas Termogênico Ir...
482885,DZSXSTAO5J,320.0,16.0,11.0,2.0,NaN,H53U1H7Q5G,NaN,NaN,NaN,MMC9EXVO9E,inactive,Scanner Universal Portugues Veiculos Alcool Ga...
146628,E8MWR6YE82,430.0,25.0,11.0,5.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-PERFUMES,296.00,LEE8IQPCSB,active,Perfume Burberry Feminino 100ml Eau De Parfum ...
48741,GGK3LKE5F1,NaN,NaN,NaN,NaN,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-MAKEUP_BRUSHES,59.90,L08IIE84TR,active,Kit Com 32 Pincéis Maquiagem + Estojo Pronta E...
39254,KVZ02T28SV,750.0,37.0,37.0,5.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-AUTOMOTIVE_WHEEL_COVERS,26.49,D78NNMN2B1,paused,Jogo 4 Calota Centro Miolo Roda Esportiva Aran...
36091,QY0AH5MI7P,360.0,44.0,19.0,5.0,"[{'id': 'BACKLIGHT', 'name': 'Teclado iluminad...",WWIT2JLP2L,new,MLB-LAPTOP_KEYBOARDS,59.49,W3XL09SHF8,active,Teclado Positivo Unique S1991 82r-14e238-4211 ...
214486,PTIMWEEK2L,65.0,30.0,30.0,5.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-NECKLACES,16.71,LFPRBZWHFG,active,Escapulário São Jorge Guerreiro Aço Inox Prat...
370762,ECCI2VSOF8,1240.0,29.0,21.6,17.2,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-MUGS,59.90,ND4HY71XYQ,active,Caneca Caveira Viking Copo Choop Cerveja Crâni...
41212,ZG8R5MERFE,101.0,16.0,15.0,5.0,"[{'id': 'ITEM_CONDITION', 'name': 'Condição do...",H53U1H7Q5G,new,MLB-BRACELETS,129.00,BUBEV02ZT5,paused,Pulseira Infantil Em Ouro 18k/750 C/ Pérolas -...
162906,ZPHFGU2OJ8,4500.0,30.0,30.0,20.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-CARPETS,180.00,TTJLWEOJGA,paused,Tapete Muito Peludo Redondo 4cm Shaggy Luxo Di...


In [25]:
df = df.dropna(subset=['SHP_WEIGHT', 'SHP_LENGTH', 'SHP_WIDTH', 'SHP_HEIGHT'])
##df = df.dropna(subset=['SHP_LENGTH'])
##df = df.dropna(subset=['SHP_WIDTH'])
##df = df.dropna(subset=['SHP_HEIGHT'])
df = df[ (df['SHP_WEIGHT'] > 0) & (df['SHP_LENGTH'] >  0) & (df['SHP_WIDTH'] >  0) & (df['SHP_HEIGHT'] >0)]
df = df[ (df['SHP_WEIGHT'] < 100) & (df['SHP_LENGTH'] <  100) & (df['SHP_WIDTH'] <  100) & (df['SHP_HEIGHT'] < 100)]
df.sample(300)

,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,ATTRIBUTES,CATALOG_PRODUCT_ID,CONDITION,DOMAIN_ID,PRICE,SELLER_ID,STATUS,TITLE
475760,OWWQDTUG54,70.0,16.0,11.0,4.0,"[{'id': 'AGE_GROUP', 'name': 'Idade', 'value_i...",H53U1H7Q5G,new,MLB-PERFUMES,40.00,SDFB6FE1L6,active,2 Atomizador Porta Perfume Travalo Viagem Deca...
195307,ZKI6QZ1U1N,80.0,17.0,13.0,4.0,NaN,H53U1H7Q5G,NaN,NaN,NaN,NaN,404,NaN
356431,UINEFIJLNN,30.0,16.0,16.0,4.0,"[{'id': 'BRAND', 'name': 'Marca da película', ...",H53U1H7Q5G,new,MLB-CELLPHONE_AND_TABLET_SCREEN_PROTECTORS,52.99,BIOKXZ6LOT,active,Película De Gel Adere Na Curva Motorola Moto Z...
237358,HXKYXXBT0R,60.0,16.0,11.0,4.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-BABY_SUPPLIES,58.00,ZIE10PJRD1,active,Travesseiro Preventor De Cabeça Chata Para Beb...
156979,ZAXP81L9YC,50.0,25.0,11.0,5.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-SIM_CARD_TRAYS,21.99,IWHQTXPRVC,active,Gaveta Bandeja Slot Sim Card Chip Sony Xperia ...
132794,D71F9DQG60,42.0,40.0,20.0,10.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-BICYCLE_ACCESSORIES_AND_SPARE_PARTS,39.00,YHMH8IWKS9,active,Selim Gios Gi-1109 Courvin Preto Trilho Aço Lo...
248541,DYX4JP0AKM,40.0,16.0,11.0,4.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-CELLPHONE_REPLACEMENT_KEYBOARDS,19.90,VZVTOHIQ8F,active,Botão Externo Power E Volume Moto G4 Play Xt1600
460389,CRPP6HOA0Z,53.0,25.0,20.0,15.0,NaN,H53U1H7Q5G,NaN,NaN,NaN,NaN,404,NaN
300138,DJX5CQWHBQ,34.0,32.0,23.0,12.0,NaN,H53U1H7Q5G,NaN,NaN,NaN,NaN,404,NaN
369199,HJ04I90UDD,53.0,16.0,14.0,6.0,NaN,H53U1H7Q5G,NaN,NaN,NaN,NaN,404,NaN
